## Processing T1w images to estimate Gray Matter Density


In [ ]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm.preprocess import VBMSegment, Segment
from nipype.interfaces.ants import Atropos, Registration, ApplyTransforms, N4BiasFieldCorrection
from nipype.interfaces.fsl import ApplyMask, BET
from pandas import DataFrame, Series, read_csv

# Study specific variables
study_home = '/moochie/Cat/Aggregate_anats/GMD_ML'

sub_data_file = study_home + '/doc/subjectinfo.csv'
subject_info = read_csv(sub_data_file)
subjects_list = subject_info['SubjectID'].tolist()
#subjects_list = ['101']

raw_data = study_home + '/raw'
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'

sample_template = study_home + '/templates/lcbd_template_1mm.nii.gz'
sample_template_brain = study_home + '/templates/lcbd_template_1mm_brain.nii.gz'
sample_template_mask = study_home + '/templates/lcbd_template_1mm_mask.nii.gz'

# temporary tissue priors
warp_gm = study_home + '/templates/MNI_gray_warp.nii.gz'
warp_wm = study_home + '/templates/MNI_white_warp.nii.gz'
warp_csf = study_home + '/templates/MNI_csf_warp.nii.gz'

In [ ]:
# Data handling nodes
infosource = Node(IdentityInterface(fields=['subjid']), 
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

substitutions = [('_subjid_', '')]
datasink = Node(DataSink(substitutions=substitutions, 
                         base_directory=output_dir,
                         container=output_dir), 
                name='datasink')

### Bias-correct and segment subject images

In [ ]:
# select the appropriate file to segment
segmenttemplate = {'anat': raw_data + '/{subjid}/mprage.nii.gz'}
segmentfiles = Node(SelectFiles(segmenttemplate), name='segmentfiles')

# N4 bias correction
biascorrect = Node(N4BiasFieldCorrection(save_bias=True),name='biascorrect')

# Register to template brain
reg_temp1 = Node(Registration(fixed_image=sample_template,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=1,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Rigid', 'Affine'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp1')

# Skullstripping
#skullstrip = Node(ApplyMask(mask_file=sample_template_mask),name='skullstrip')
skullstrip = Node(BET(),name='skullstrip')

# Register to skullstripped brain
reg_temp2 = Node(Registration(fixed_image=sample_template_brain,
                              args='--float',
                              collapse_output_transforms=True,
                              initial_moving_transform_com=True,
                              num_threads=1,
                              output_inverse_warped_image=True,
                              output_warped_image=True,
                              sigma_units=['vox']*2,
                              transforms=['Affine', 'SyN'],
                              terminal_output='file',
                              winsorize_lower_quantile=0.005,
                              winsorize_upper_quantile=0.995,
                              convergence_threshold=[1e-06],
                              convergence_window_size=[10],
                              metric=['Mattes', 'Mattes'],
                              metric_weight=[1.0]*2,
                              number_of_iterations=[[100, 75, 50],
                                                    [100, 75, 50]],
                              radius_or_number_of_bins=[32, 32],
                              sampling_percentage=[0.25, 0.25],
                              sampling_strategy=['Regular',
                                                 'Regular'],
                              shrink_factors=[[4, 2, 1]]*2,
                              smoothing_sigmas=[[2, 1, 0]]*2,
                              transform_parameters=[(0.1,),
                                                    (0.1,)],
                              use_histogram_matching=False,
                              write_composite_transform=True),
                 name='reg_temp2')

# Atropos segmentation
segment = Node(Atropos(initialization='Otsu',
                       number_of_tissue_classes=3, 
                       mask_image=sample_template_mask, 
                       save_posteriors=True, n_iterations=10),name='segment')

In [ ]:
segproc = Workflow(name='segproc')
segproc.connect([(infosource, segmentfiles,[('subjid','subjid')]), 
                 (segmentfiles,biascorrect,[('anat','input_image')]),
                 (biascorrect,reg_temp1,[('output_image','moving_image')]),
                 (reg_temp1, skullstrip, [('warped_image','in_file')]),
                 (skullstrip, reg_temp2, [('out_file','moving_image')]), 
                 (reg_temp2, segment, [('warped_image','intensity_images')]),
                 
                 (reg_temp1, datasink, [('composite_transform','reg1_composite_transform'), 
                                        ('inverse_composite_transform','reg1_inverse_composite_transform'),
                                        ('warped_image','reg1_warped_anat')]),
                 (reg_temp2, datasink, [('composite_transform','reg2_composite_transform'),
                                        ('inverse_composite_transform','reg2_inverse_composite_transform'),
                                        ('warped_image','reg2_warped_anat')]),
                 (segment, datasink, [('classified_image','hard_tissue_files'),('posteriors','soft_tissue_files')])
                 
                ])
segproc.base_dir = workflow_dir
segproc.write_graph(graph2use='flat')
segproc.run('MultiProc', plugin_args={'n_procs': 4,'memory_gb':10})